In [1]:
import argparse
import os
import time
import pytorch_lightning as pl
import torchvision.transforms
from rich import print
from torch.utils.data import DataLoader
import lima
import glob
import h5py
import skimage.io as io
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import numpy as np
import cmcrameri.cm as cmc
from pykitPIV import Particle, FlowField, Motion, Image

lvpyio module is not available on macOS.


/Users/kamilazdybal/anaconda3/envs/lima-pykitPIV/lib/python3.10/site-packages/lima/__init__.py:14: UserWarning: GPU not available and could not be checked!
  warn("GPU not available and could not be checked!")
2025-05-22 13:16:38.159172: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
save_images = False
filename = None

In [5]:
figsize = (5,3)

<a id=train-LIMA></a>
***

## Train `LIMA` with the generated images

[Go to the top](#top-page)

<a id=train-LIMA-input-data></a>
### Prepare input dataset for LIMA

[Go to the top](#top-page)

In [47]:
transform = torchvision.transforms.Compose([lima.datatransform.RandomAffine(degrees=17, translate=(0.2, 0.2), scale=(0.9, 2.0)),
                                            lima.datatransform.RandomHorizontalFlip(),
                                            lima.datatransform.RandomVerticalFlip(),
                                            lima.datatransform.ToTensor(),
                                            lima.datatransform.RandomBrightness(factor=(0.5, 2)),
                                            lima.datatransform.RandomNoise(std=(0, 0)),])

#### Use dataset generated with `pykitPIV`:

[Go to the top](#top-page)

In [ ]:
# class pykitPIVDataset(Dataset):
#     """Load pykitPIV-generated dataset"""

#     def __init__(self, image_pairs, targets, transform=None, n_samples=None, pin_to_ram=False):

#         self.data = image_pairs.astype(np.float32)
#         self.target = targets.astype(np.float32)

#         if n_samples:
#             self.data = self.data[:n_samples]
#             self.target = self.target[:n_samples]
#         if pin_to_ram:
#             self.data = np.array(self.data)
#             self.target = np.array(self.target)

#         self.transform = transform

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
#         sample = self.data[idx], self.target[idx]
#         if self.transform:
#             sample = self.transform(sample)

#         return sample

In [ ]:
# train_dataset = pykitPIVDataset(image_pairs=image_pairs_train,
#                                 targets=targets_train,
#                                 transform=transform)

In [ ]:
# test_dataset = pykitPIVDataset(image_pairs=image_pairs_test,
#                                 targets=targets_test,
#                                 transform=transform)

In [ ]:
class pykitPIVDatasetFromPath(Dataset):
    """Load pykitPIV-generated dataset"""

    def __init__(self, path, transform=None, n_samples=None, pin_to_ram=False):
        
        f = h5py.File(path, "r")
        self.data = f["I"]
        self.target = np.array(f["targets"])

        print(self.target.max())

        if n_samples:
            self.data = self.data[:n_samples]
            self.target = self.target[:n_samples]
            
        if pin_to_ram:
            self.data = np.array(self.data)
            self.target = np.array(self.target)
            f.close()
            
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        sample = self.data[idx], self.target[idx]
        
        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
train_dataset = pykitPIVDatasetFromPath(path='PIV-dataset-train.h5',
                                        transform=transform)

In [ ]:
train_dataset.data

In [ ]:
test_dataset = pykitPIVDatasetFromPath(path='PIV-dataset-test.h5',
                                       transform=transform)

#### Use dataset generated with Matlab:

[Go to the top](#top-page)

In [ ]:
# path = 'PIV_n3_s180_maxd10_rnd_v1.h5'

In [ ]:
# f = h5py.File(path, "r")

# images = f["I"]
# images = np.array(images)
# targets = f["target"]
# targets = np.array(targets)[:,2:4,:,:]

# f.close()

In [ ]:
# plt.imshow(np.array(images)[0,0,:,:], cmap='Greys_r')

In [ ]:
# class HDF5Dataset(Dataset):
#     """HDF5Dataset loaded"""

#     def __init__(self, path, transform=None, n_samples=None, pin_to_ram=False):
#         f = h5py.File(path, "r")
#         self.data = f["I"]
#         self.target = np.array(f["target"])[:,2:4,:,:]

#         if n_samples:
#             self.data = self.data[:n_samples]
#             self.target = self.target[:n_samples]
#         if pin_to_ram:
#             self.data = np.array(self.data)
#             self.target = np.array(self.target)
#             f.close()
#         self.transform = transform

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
#         sample = self.data[idx], self.target[idx]
#         if self.transform:
#             sample = self.transform(sample)

#         return sample

In [ ]:
# train_dataset = HDF5Dataset(path=path,
#                             transform=transform,)

In [ ]:
# test_dataset = HDF5Dataset(path=path,
#                            transform=transform,)

***

© K. Zdybał, C. Mucignat, S. Kunz, I. Lunati (2025)